In [1]:
#1/423536400
from math import fabs
import openpyxl
from ortools.linear_solver import pywraplp
import random
from openpyxl import Workbook, load_workbook
import numpy as np
import  plotly
import plotly.graph_objs as go
import os,sys
solver = pywraplp.Solver.CreateSolver('GLOP')
def solve(a,b,c1, c2):
    x = solver.NumVar(0,50,'x')
    y = solver.NumVar(0,50,'y')
    constr1 = solver.Constraint(0,b[0],'ct')
    constr1.SetCoefficient(x,a[0])
    constr1.SetCoefficient(y,a[1])
    constr2 = solver.Constraint(0,b[1],'ct')
    constr2.SetCoefficient(x,a[2])
    constr2.SetCoefficient(y,a[3])
    obje = solver.Objective()
    obje.SetCoefficient(x, c1)
    obje.SetCoefficient(y, c2)
    obje.SetMaximization()
    solver.Solve()
    return c1*x.solution_value()+c2*y.solution_value()
def setparam(a,b,cf,param_a,param_b, c1, c2):
    dia1 = ""
    dia2 = ""
    a[param_a] -= 1
    if cf > solve(a,b,c1,c2):
        a[param_a] += 1
        dia1 += "[" + str(a[param_a]-8) + ".." + str(a[param_a]) + "]"

    else:
        a[param_a] += 2
        if cf > solve(a,b,c1,c2):
            a[param_a] -= 1
            dia1 += "[" + str(a[param_a]) + ".." + str(a[param_a]+8) + "]"

        else:
            return ""

    b[param_b] -= 1
    if cf > solve(a,b,c1,c2):
        b[param_b] += 1
        dia2 += "[" + str(b[param_b]- 8) + ".." +  str(b[param_b]) + "]"
    else:
        b[param_b] += 2
        if cf > solve(a,b,c1,c2):
            b[param_b] -= 1
            dia2 += "[" + str(b[param_b]) + ".." + str(b[param_b]+8) + "]"
        else:
            return ""

    return "a: " + dia1 + " || b: " + dia2

def f(x,a,b,k):
    if k == 0:
        return (b[0]-a[0]*x)/a[1]
    else:
        return (b[1]-a[2]*x)/a[3]
try:
    os.remove("1.xlsx")
except:
    key = 0
key = 0
numel = 0
wb = openpyxl.Workbook()
while(True):
    try:
        print("Enter diapasone for c  : ")
        cd = int(input())
        print("Enter diapasone for a  : ")
        ad = int(input())
        print("Enter diapasone for b (for correct work b must be > 2) : ")
        bd = int(input())
        print("Enter num of expression  : ")
        numel = int(input())
        break
    except:
        print("Please, input correct data(integer value)")

while(key != numel):
    constraints_num = 2

    x1 = solver.NumVar(0,50,'x1')# variable creating
    x2 = solver.NumVar(0,50,'x2')

    b1 = random.randrange(2,bd,1)
    ct1 = solver.Constraint(0,b1,'ct')

    while(True):
        a1 = random.randrange(1,ad,1)
        a2 = random.randrange(-ad,ad,1)
        a3 = random.randrange(1,ad,1)
        if a2 < 0:
            a4 = random.randrange(1,ad,1)
        else:
            a4 = random.randrange(-ad,ad,1)
        if a2 != 0 and a4 != 0 :
            break
    ct1.SetCoefficient(x1,a1)
    ct1.SetCoefficient(x2,a2)
    b2 = random.randrange(2,bd,1)
    ct2 = solver.Constraint(0,b2,'ct')

    ct2.SetCoefficient(x1,a3)

    ct2.SetCoefficient(x2,a4)

    c1 = random.randrange(1,cd,1)
    c2 = random.randrange(1,cd,1)
    obj = solver.Objective()
    obj.SetCoefficient(x1, c1)
    obj.SetCoefficient(x2, c2)
    obj.SetMaximization()
    solver.Solve()
    xx1 = x1.solution_value()
    xx2 = x2.solution_value()

    if  fabs(round(xx1)-xx1) < 0.0000000001 and fabs(round(xx2)-xx2) < 0.0000000001 :
        cf = c1*xx1+c2*xx2
        a = [a1,a2,a3,a4]
        b = [b1,b2]
        param_a = random.randrange(0,4,1)
        if param_a < 2:
            param_b = 1
        else:
            param_b = 0
        res = setparam(a,b,cf,param_a,param_b, c1, c2)

        try:
            point = [-(b[0]*a[3] - b[1]*a[1])/(a[2]*a[1] - a[0]*a[3]),
                    (b[1] + (b[0]*a[3] - b[1]*a[1])/(a[2]*a[1] - a[0]*a[3])*a[2])/a[3]
                    ]
        except:
            continue

        if res == "" or ( fabs(round(point[0])-point[0]) > 0.0000000001 or fabs(round(point[1])-point[1]) > 0.0000000001 or (point[0] < 0 and point[1] < 0)) :
            continue
        else:


            x = np.arange(0,15,1)
            # fig = go.Figure()
            # fig.add_trace(go.Scatter(x=x, y=f(x,a,b,0)))
            # fig.add_trace(go.Scatter(x=x, y=f(x,a,b,1)))
            # fig.show()


            ws = wb.create_sheet(str(key))
            param = [a[param_a],b[param_b]]
            a[param_a] = "a"
            b[param_b] = "b"
            ws['A1'].value = str(c1)+"x1 + "+str(c2)+"x2 -> Max"
            ws['A2'].value = str(a[0])+"x1 + "+str(a[1])+"x2 <= "+ str(b[0])
            ws['A3'].value = str(a[2])+"x1 + "+str(a[3])+"x2 <= "+ str(b[1])
            ws['A4'].value = 'Solution:'
            ws['A5'].value = 'Objective value = ' + str(round(cf))
            ws['A6'].value = 'x1 =' + str(round(xx1))
            ws['A7'].value = 'x2 =' + str(round(xx2))
            ws['A8'].value = "a: " + str(param[0]) + " b: " + str(param[1])
            ws['A9'].value = res

            print(str(point[0])+ " " + str(point[1]))
            print(str(c1)+"x1 + "+str(c2)+"x2 -> Max")
            print(str(a[0])+"x1 + "+str(a[1])+"x2 <= "+ str(b[0]))
            print(str(a[2])+"x1 + "+str(a[3])+"x2 <= "+ str(b[1]))
            print('Solution:')
            print('Objective value =', round(cf))
            print('x1 =', round(xx1))
            print('x2 =', round(xx2))

            print("a: "+str(param[0]) + " b: " + str(param[1]))
            print(res)
            key += 1
            print(str(key)+"\n")
sheet = wb.get_sheet_by_name('Sheet')
wb.remove_sheet(sheet)
wb.save("1.xlsx")
os.system("1.xlsx")


Enter diapasone for c  : 
Enter diapasone for a  : 
Enter diapasone for b (for correct work b must be > 2) : 
Enter num of expression  : 
2.0 1.0
1x1 + 1x2 -> Max
3x1 + 4x2 <= b
ax1 + 1x2 <= 9
Solution:
Objective value = 3
x1 = 2
x2 = 1
a: 4 b: 10
a: [4..12] || b: [2..10]
1

7.0 0.0
1x1 + 1x2 -> Max
ax1 + -1x2 <= 7
1x1 + 2x2 <= b
Solution:
Objective value = 7
x1 = 7
x2 = 0
a: 1 b: 7
a: [1..9] || b: [-1..7]
2

4.0 3.0
1x1 + 1x2 -> Max
ax1 + 2x2 <= 10
1x1 + 1x2 <= b
Solution:
Objective value = 7
x1 = 7
x2 = 0
a: 1 b: 7
a: [-7..1] || b: [-1..7]
3

1.0 2.0
1x1 + 1x2 -> Max
1x1 + 1x2 <= b
3x1 + ax2 <= 5
Solution:
Objective value = 3
x1 = 0
x2 = 3
a: 1 b: 3
a: [1..9] || b: [-5..3]
4

2.0 1.0
1x1 + 1x2 -> Max
1x1 + 4x2 <= b
ax1 + -3x2 <= 5
Solution:
Objective value = 3
x1 = 2
x2 = 1
a: 4 b: 6
a: [4..12] || b: [-2..6]
5

2.0 0.0
1x1 + 1x2 -> Max
3x1 + 2x2 <= b
2x1 + ax2 <= 4
Solution:
Objective value = 3
x1 = 0
x2 = 3
a: 1 b: 6
a: [1..9] || b: [-2..6]
6

3.0 0.0
1x1 + 1x2 -> Max
ax1 + -5x2 <= 

c:\users\lenovo\pycharmprojects\pythonproject3\venv\lib\site-packages\ipykernel_launcher.py:179: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
c:\users\lenovo\pycharmprojects\pythonproject3\venv\lib\site-packages\ipykernel_launcher.py:180: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


0